# <span style="font-family: Arial, sans-serif; color:#97f788">xbooster</span>

<span style="font-family: Arial, sans-serif; color:navyblue">Repo: <a href="https://github.com/xRiskLab/xBooster" title="GitHub link">https://github.com/xRiskLab/xBooster</a></span>

## <span style="font-family: Arial, sans-serif; color:navyblue">Scorecard by Intervals</span>

<span style="font-family: Arial, sans-serif; color:navyblue">Author: <a href="https://github.com/jmonteroers" title="GitHub link">https://github.com/jmonteroers</a></span>

This short notebook illustrates how to transform the standard XBooster scorecard into a practical scorecard by intervals of the features. In essence, it combines the different decision rules from weak learners in the XGBoost model into intervals as per the industry standard (Siddiqi, 2017). Lastly, we show how to add points by the Points at Even Odds/Points to Double the Odds method (PEO/PDO) to this latter scorecard.

Please note that for the method to work, the XGBoost model fitted must have maximum depth one (that is, the weak learners must be tree stumps), and currently, points must have been previously calculated using `XGBScorecardConstructor.create_points()` (see below).


### Set-up


The set-up replicates that in the notebook 'Getting started', obtaining a standard XBooster scorecard.

The first step is to load the data.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Fetch blended credit data
url = "https://github.com/xRiskLab/xBooster/raw/main/examples/data/credit_data.parquet"
dataset = pd.read_parquet(url)

features = [
    "external_risk_estimate",
    "revolving_utilization_of_unsecured_lines",
    "account_never_delinq_percent",
    "net_fraction_revolving_burden",
    "num_total_cc_accounts",
    "average_months_in_file",
]

target = "is_bad"

X, y = dataset[features], dataset[target]

ix_train, ix_test = train_test_split(X.index, stratify=y, test_size=0.3, random_state=62)

Next, we train the XGBoost model using the train dataset.


In [3]:
import xgboost as xgb

best_params = dict[str, float | str](
    n_estimators=100,
    learning_rate=0.55,
    # NOTE: max_depth=1 is required!
    max_depth=1,
    min_child_weight=10,
    grow_policy="lossguide",
    early_stopping_rounds=5,
)

# Create an XGBoost model
xgb_model = xgb.XGBClassifier(**best_params, random_state=62)
evalset = [
    (X.loc[ix_train], y.loc[ix_train]),
    (X.loc[ix_test], y.loc[ix_test]),
]

# Fit the XGBoost model
xgb_model.fit(
    X.loc[ix_train],
    y.loc[ix_train],
    eval_set=evalset,
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy='lossguide',
              importance_type=None, interaction_constraints=None,
              learning_rate=0.55, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=1,
              max_leaves=None, min_child_weight=10, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

Now, thanks to `xBooster`, constructing a scorecard by decision rules is as simple as:


In [4]:
from xbooster.constructor import XGBScorecardConstructor

# Set up the scorecard constructor
scorecard_constructor = XGBScorecardConstructor(xgb_model, X.loc[ix_train], y.loc[ix_train])

scorecard_by_dr = scorecard_constructor.construct_scorecard()
# Add points
xgb_scorecard_with_points = scorecard_constructor.create_points(
    pdo=50, target_points=600, target_odds=50
)

xgb_scorecard_with_points.head()

,Tree,Node,Feature,Sign,Split,Count,CountPct,NonEvents,Events,EventRate,WOE,IV,XAddEvidence,DetailedSplit,Points
0,0,1,account_never_delinq_percent,<,98.000000,2810.0,0.401429,2136.0,674.0,0.239858,1.043764,0.651110,0.851319,account_never_delinq_percent < 98,-2
1,0,2,account_never_delinq_percent,>=,98.000000,4190.0,0.598571,4164.0,26.0,0.006205,-2.878910,1.795892,-0.571674,account_never_delinq_percent >= 98 or missing,101
2,1,1,revolving_utilization_of_unsecured_lines,<,0.609306,5224.0,0.746286,4986.0,238.0,0.045559,-0.844894,0.381409,-0.388105,revolving_utilization_of_unsecured_lines < 0.6...,71
3,1,2,revolving_utilization_of_unsecured_lines,>=,0.609306,1776.0,0.253714,1314.0,462.0,0.260135,1.151958,0.520027,0.619268,revolving_utilization_of_unsecured_lines >= 0....,-2
4,2,1,external_risk_estimate,<,70.000000,2072.0,0.296000,1610.0,462.0,0.222973,0.948800,0.383737,0.457512,external_risk_estimate < 70,-2


Next, we combine decision rules into intervals, leading to a more compact version of a scorecard.


### Scorecard by Intervals

Using the same constructor, we are just one step away to build a compact scorecard grouped by intervals:


In [5]:
xgb_scorecard_by_intervals = scorecard_constructor.construct_scorecard_by_intervals()
xgb_scorecard_by_intervals.head()

,Feature,Bin,Left,Right,Points,XAddEvidence,Count,Events,NonEvents,CountPct,WOE,IV
0,account_never_delinq_percent,"(-inf, 70.8000031)",-inf,70.800003,-24,1.992827,237.0,102.0,135.0,0.033857,1.916923,0.039708
1,account_never_delinq_percent,"[70.8000031, 80.5)",70.800003,80.500000,-9,1.788010,305.0,100.0,205.0,0.043571,1.479385,0.027200
2,account_never_delinq_percent,"[80.5, 87.9000015)",80.500000,87.900002,19,1.401378,524.0,139.0,385.0,0.074857,1.178455,0.026998
3,account_never_delinq_percent,"[87.9000015, 88.5999985)",87.900002,88.599998,33,1.212278,54.0,11.0,43.0,0.007714,0.833920,0.001235
4,account_never_delinq_percent,"[88.5999985, 93.8000031)",88.599998,93.800003,39,1.136591,763.0,169.0,594.0,0.109000,0.940244,0.023058


Let us compare the length of each scorecard:


In [9]:
print(f"Length of the scorecard by decision rules: {len(xgb_scorecard_with_points)}")
print(f"Length of the scorecard by intervals: {len(xgb_scorecard_by_intervals)}")

Length of the scorecard by decision rules: 170
Length of the scorecard by intervals: 54


The latter format is more than three times shorter than the original! So now it is really much easier to use and interpret.


Finally, let us deal with the issue of negative points by directly using the PEO/PDO method on this scorecard:


In [6]:
scorecard_constructor.create_points_peo_pdo(peo=600, pdo=50).head()

,Feature,Bin,Left,Right,Points,XAddEvidence,Count,Events,NonEvents,CountPct,WOE,IV,Points_PEO_PDO
0,account_never_delinq_percent,"(-inf, 70.8000031)",-inf,70.800003,-24,1.992827,237.0,102.0,135.0,0.033857,1.916923,0.039708,-17.0
1,account_never_delinq_percent,"[70.8000031, 80.5)",70.800003,80.500000,-9,1.788010,305.0,100.0,205.0,0.043571,1.479385,0.027200,-3.0
2,account_never_delinq_percent,"[80.5, 87.9000015)",80.500000,87.900002,19,1.401378,524.0,139.0,385.0,0.074857,1.178455,0.026998,25.0
3,account_never_delinq_percent,"[87.9000015, 88.5999985)",87.900002,88.599998,33,1.212278,54.0,11.0,43.0,0.007714,0.833920,0.001235,39.0
4,account_never_delinq_percent,"[88.5999985, 93.8000031)",88.599998,93.800003,39,1.136591,763.0,169.0,594.0,0.109000,0.940244,0.023058,44.0


In summary, now we have a compact scorecard with points in a practical scale - only using two additional methods of the `XGBScorecardConstructor` constructor.
